# **Holiday Package Prediction**

**Steps of the Data Science Pipeline**
- Acquire our Data
- Prepare/Clean our Data
- Explore/Analyze our Data
- Model

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import wrangle as w
import explore as ex

### Acquire Data

**Acquired data from [here](https://www.kaggle.com/susant4learning/holiday-package-purchase-prediction?select=Travel.csv)

In [30]:
# bring in CSV file
df = pd.read_csv('Travel.csv')

In [31]:
w.get_info(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4888 entries, 0 to 4887
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CustomerID                4888 non-null   int64  
 1   ProdTaken                 4888 non-null   int64  
 2   Age                       4662 non-null   float64
 3   TypeofContact             4863 non-null   object 
 4   CityTier                  4888 non-null   int64  
 5   DurationOfPitch           4637 non-null   float64
 6   Occupation                4888 non-null   object 
 7   Gender                    4888 non-null   object 
 8   NumberOfPersonVisiting    4888 non-null   int64  
 9   NumberOfFollowups         4843 non-null   float64
 10  ProductPitched            4888 non-null   object 
 11  PreferredPropertyStar     4862 non-null   float64
 12  MaritalStatus             4888 non-null   object 
 13  NumberOfTrips             4748 non-null   float64
 14  Passport

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
198,200198,0,38.0,Self Enquiry,1,8.0,Salaried,Male,2,3.0,Deluxe,3.0,Divorced,1.0,0,2,0,1.0,Manager,21553.0
1045,201045,0,40.0,Self Enquiry,1,8.0,Small Business,Male,3,3.0,Basic,3.0,Married,3.0,0,1,0,0.0,Executive,17345.0
3699,203699,1,34.0,Self Enquiry,3,23.0,Salaried,Fe Male,4,4.0,Standard,5.0,Unmarried,4.0,1,5,0,1.0,Senior Manager,27242.0


### Data Acquisition Takeaways:
 - Most of our columns are numeric types.
 - There are a handful of nulls that I need to handle.
 - Object columns will need to be addressed and be changed to numeral types.

### **Prepare/Clean Data**

In [32]:
# first and foremost let's rename our columns to more human readable

# make columns all lowercase
df.rename(str.lower, axis='columns', inplace=True)

# rename columns
df.rename(columns={'customerid': 'cust_id',
                   'prodtaken': 'product_taken',
                   'typeofcontact': 'contact_type',
                   'citytier': 'city_tier',
                   'durationofpitch': 'pitch_duration',
                   'numberofpersonvisiting': 'group_size',
                   'numberoffollowups': 'no_followups',
                   'productpitched': 'product_pitched',
                   'preferredpropertystar': 'preferred_property_star',
                   'maritalstatus': 'marital_status',
                   'numberoftrips': 'no_trips',
                   'pitchsatisfactionscore': 'pitch_sat_score',
                   'owncar': 'has_car',
                   'numberofchildrenvisiting': 'no_children',
                   'monthlyincome': 'monthly_income'
}, inplace=True)

In [33]:
# get the values for contact type column including nulls
df.contact_type.value_counts(dropna=False)

Self Enquiry       3444
Company Invited    1419
NaN                  25
Name: contact_type, dtype: int64

In [34]:
df[df.contact_type=='Self Enquiry'].contact_type.count()

3444

In [38]:
# change the entries of contact_type from Self Enquiry to Self Inquiry
df.contact_type = np.where(df.contact_type=='Self Enquiry', 'Self Inquiry', df.contact_type)

In [42]:
df.contact_type.value_counts(dropna=False)

Self Inquiry       3444
Company Invited    1419
NaN                  25
Name: contact_type, dtype: int64

In [43]:
# double check there are no duplicates
df.duplicated().any()

False

In [44]:
df.columns.tolist()

['cust_id',
 'product_taken',
 'age',
 'contact_type',
 'city_tier',
 'pitch_duration',
 'occupation',
 'gender',
 'group_size',
 'no_followups',
 'product_pitched',
 'preferred_property_star',
 'marital_status',
 'no_trips',
 'passport',
 'pitch_sat_score',
 'has_car',
 'no_children',
 'designation',
 'monthly_income']

**Gather changes into a giant prepare function**

In [ ]:
def prepare_holiday(df):
    '''
    This function takes in a the holiday data set and cleans it by lowercasing and renaming columns,

    '''
    # lowercase all column names
    df.rename(str.lower, axis='columns', inplace=True)
    
    # rename our columns to more human readable
    df.rename(columns={'prodtaken': 'product_taken',
                    'typeofcontact': 'contact_type',
                    'citytier': 'city_tier',
                    'durationofpitch': 'pitch_duration',
                    'numberofpersonvisiting': 'group_size',
                    'numberoffollowups': 'no_followups',
                    'productpitched': 'product_pitched',
                    'preferredpropertystar': 'preferred_property_star',
                    'maritalstatus': 'marital_status',
                    'numberoftrips': 'no_trips',
                    'pitchsatisfactionscore': 'pitch_sat_score',
                    'owncar': 'has_car',
                    'numberofchildrenvisiting': 'no_children',
                    'monthlyincome': 'monthly_income'
    }, inplace=True)
    
    # drop customerid as it will give us zero value
    df.drop(columns='customerid', inplace=True)
    
    # change the entries of contact_type from Self Enquiry to Self Inquiry
    df.contact_type = np.where(df.contact_type=='Self Enquiry', 'Self Inquiry', df.contact_type)

    return df

**Now we need to address the nulls**

In [ ]:
df.isnull().sum()

In [ ]:
null_list = ['age', 'pitch_duration', 'no_followups', 'preferred_property_star', 'no_trips', 'no_children', 'monthly_income']

df[null_list].sample(5)

In [ ]:
# visualize the distribution of columns to get an idea of if/how we might impute them

for i in null_list:
    sns.displot(df, x=i)
    plt.title('Distribution of '+i)
    plt.show();

**Takeaways:**
- pitch_duration, no_trips, monthly income all appear to have some outliers
    - we could address by either removing them, scaling the data (robust scaler is best), or using an algorithm that is least sensitive to outliers
- For the rest of the data, it's worth imputing based off mean and mode.
- With more time, I will attempt to re-do this project with just removing the nulls and seeing if that gives me different results.

In [ ]:
def get_lower_and_upper_bounds(df, k=1.5):
    '''
    calculates the lower and upper bound to locate outliers and displays them
    note: recommended k be 1.5
    '''
    for i in df.columns:
        if df[i].dtypes != 'object':
            quartile1, quartile3 = np.percentile(df[i], [25,75])
            IQR_value = quartile3 - quartile1
            lower_bound = (quartile1 - (k * IQR_value))
            upper_bound = (quartile3 + (k * IQR_value))
            print(f'For {i} the lower bound is {lower_bound} and  upper bound is {upper_bound}')
        
        
        
def visualize_get_lower_and_upper_bounds(df, k=1.5):
    '''
    calculates the lower and upper bound to locate outliers and displays them
    note: recommended k be 1.5
    '''
    for i in df.columns:
        if df[i].dtypes != 'object':
            quartile1, quartile3 = np.percentile(df[i], [25,75])
            IQR_value = quartile3 - quartile1
            lower_bound = (quartile1 - (k * IQR_value))
            upper_bound = (quartile3 + (k * IQR_value))
            print(f'For {i} the lower bound is {lower_bound} and  upper bound is {upper_bound}')
            
            # get those visualizations going
            plt.figure(figsize=(16,4))
            plt.subplot(1, 2, 1)
            sns.histplot(data = df, x = df[i], kde=True)
            plt.title(i)
            plt.subplot(1, 2, 2)
            sns.boxplot(x=df[i], data=df, whis=k)
            plt.title(i)
            plt.show()

In [ ]:
df.pitch_duration

In [ ]:
quartile1, quartile3 = np.percentile(df.pitch_duration, [25,75])
IQR_value = quartile3 - quartile1
lower_bound = (quartile1 - (1.5 * IQR_value))
upper_bound = (quartile3 + (1.5 * IQR_value))
print(f'For pitch duration the lower bound is {lower_bound} and  upper bound is {upper_bound}')

print(quartile1, quartile3, IQR_value)

np.percentile(df.pitch_duration, 25)

In [ ]:
df.pitch_duration

In [ ]:
get_lower_and_upper_bounds(df, k=1.5)

In [ ]:
# use visualize outlier function to gauge outliers
visualize_get_lower_and_upper_bounds(df, k=1.5)

**Columns with outliers:** \
    - monthly_income\
    - no_followsup\
    - no_trips\
    - group_size\
    - pitch_duration

In [ ]:
# dive into columns with outliers
outliers = ['monthly_income', 'no_followups', 'no_trips', 'group_size', 'pitch_duration']

visualize_get_lower_and_upper_bounds(df[outliers], k=1.5)

In [ ]:
# let's take a look at the specific anomalies
# monthly income anomaly
df[df.monthly_income>50000]

In [ ]:
df[(df.no_followups>5) | (df.no_followups<2)].no_followups.value_counts()

In [ ]:
df[df.no_trips>=7].no_trips.value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
missing_values_table(df)

In [ ]:
# why are the columns printing out weird
# I suspect there may be some trailing spaces
df.columns

In [ ]:
df.columns.tolist()

In [ ]:
for i in df.columns:
    df[i] = df[i.strip()]

In [ ]:
df.columns = df.columns.str.replace(' ', '')

In [ ]:
df.columns

In [ ]:
df.head()

**Analyze Nulls**

In [ ]:
df.isnull().sum()

In [ ]:
sns.distplot(df.age);

In [ ]:
sns.distplot(df.pitch_duration);